In [1]:
import psutil
import platform
from datetime import datetime

In [2]:
def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

In [3]:
print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

======================================== System Information ========================================
System: Windows
Node Name: DESKTOP-C67QDCR
Release: 10
Version: 10.0.19041
Machine: AMD64
Processor: Intel64 Family 6 Model 142 Stepping 10, GenuineIntel


In [4]:
# Boot Time
print("="*40, "Boot Time", "="*40)
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

======================================== Boot Time ========================================
Boot Time: 2021/10/21 16:47:0


In [5]:
# let's print CPU information
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 1800.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 1600.00Mhz
CPU Usage Per Core:
Core 0: 6.2%
Core 1: 3.1%
Core 2: 21.5%
Core 3: 3.1%
Core 4: 4.7%
Core 5: 3.1%
Core 6: 4.7%
Core 7: 3.1%
Total CPU Usage: 9.3%


In [6]:
# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

======================================== Memory Information ========================================
Total: 11.86GB
Available: 4.31GB
Used: 7.55GB
Percentage: 63.6%
==================== SWAP ====================
Total: 13.67GB
Free: 4.73GB
Used: 8.94GB
Percentage: 65.4%


In [7]:
# Disk Information
print("="*40, "Disk Information", "="*40)
print("Partitions and Usage:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    print(f"  File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_size(partition_usage.total)}")
    print(f"  Used: {get_size(partition_usage.used)}")
    print(f"  Free: {get_size(partition_usage.free)}")
    print(f"  Percentage: {partition_usage.percent}%")
# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

======================================== Disk Information ========================================
Partitions and Usage:
=== Device: C:\ ===
  Mountpoint: C:\
  File system type: NTFS
  Total Size: 237.97GB
  Used: 228.89GB
  Free: 9.09GB
  Percentage: 96.2%
Total read: 19.48GB
Total write: 19.01GB


In [8]:
# Network information
print("="*40, "Network Information", "="*40)
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

======================================== Network Information ========================================
=== Interface: Ethernet ===
=== Interface: Ethernet ===
  IP Address: 169.254.182.139
  Netmask: 255.255.0.0
  Broadcast IP: None
=== Interface: Ethernet ===
=== Interface: vEthernet (WSL) ===
=== Interface: vEthernet (WSL) ===
  IP Address: 172.25.128.1
  Netmask: 255.255.240.0
  Broadcast IP: None
=== Interface: vEthernet (WSL) ===
=== Interface: Conexión de área local* 1 ===
=== Interface: Conexión de área local* 1 ===
  IP Address: 169.254.30.84
  Netmask: 255.255.0.0
  Broadcast IP: None
=== Interface: Conexión de área local* 1 ===
=== Interface: Conexión de área local* 10 ===
=== Interface: Conexión de área local* 10 ===
  IP Address: 169.254.96.138
  Netmask: 255.255.0.0
  Broadcast IP: None
=== Interface: Conexión de área local* 10 ===
=== Interface: Wi-Fi ===
=== Interface: Wi-Fi ===
  IP Address: 192.168.100.100
  Netmask: 255.255.255.0
  Broadcast IP: None
=== Interface: Wi-